In [1]:
import os
from bs4 import BeautifulSoup
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
import warnings
from langchain_community.document_loaders import DirectoryLoader
from llama_index.core import SimpleDirectoryReader, StorageContext
warnings.filterwarnings("ignore")
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
import numpy as np
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
from llmlingua import PromptCompressor
import re
from IPython.core.display import display, HTML

Package protobuf is installed but has a version conflict:
	(protobuf 3.20.3 (/opt/anaconda3/lib/python3.11/site-packages), Requirement.parse('protobuf>=4.23.2'))

This package is optional for trulens_eval so this may not be a problem but if
you need to use the related optional features and find there are errors, you
will need to resolve the conflict:

    ```bash
    pip install 'protobuf>=4.23.2'
    ```

If you are running trulens_eval in a notebook, you may need to restart the
kernel after resolving the conflict. If your distribution is in a bad place
beyond this package, you may need to reinstall trulens_eval so that all of the
dependencies get installed and hopefully corrected:
    
    ```bash
    pip uninstall -y trulens_eval
    pip install trulens_eval
    ```

Package watchdog is installed but has a version conflict:
	(watchdog 2.1.6 (/opt/anaconda3/lib/python3.11/site-packages), Requirement.parse('watchdog>=3.0.0'))

This package is optional for trulens_eval so this may not 

In [2]:
NEO4J_URI = 'bolt://' + 'localhost' + ':7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = '12345678'
NEO4J_DATABASE = 'neo4j'

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

### Grab data from federal law website

In [214]:


def grab_link_from_html(url, element, class_name):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    # return the href attribute of all in an array
    return [a['href'].split("/")[0] for a in soup.find_all(element, class_=class_name)]

In [168]:
# find the link to "Purpose of the act" and click

def grab_purpose_of_act_url(url):
    print(url)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    # find spam with value "Purpose of the act"
    span = soup.find_all('span', string="Purpose of Act")
    # grab href of the parent of the span
    if span == []:
        span = soup.find_all('span', string="Short Title")

    return span[0].parent['href']
    
    

In [ ]:


acts_index = [ 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y']

gc_url = "https://laws-lois.justice.gc.ca/eng/regulations/" # /acts/ for acts
gc_xml_url = "https://laws-lois.justice.gc.ca/eng/XML/"

for act in acts_index:
    act_list_url = gc_url + act + ".html"
    # get all the links within a span class "objTitle"
    act_tags = grab_link_from_html(act_list_url, 'a', "TocTitle")
    
    for tag in act_tags:
        xml_url = gc_xml_url + tag + '.xml'
        law_url = gc_url + tag + '/page-1.html'
        extract_text_from_xml(xml_url, law_url)
    
    

In [289]:
import urllib
def extract_text_from_xml(url, law_url):
    response = urllib.request.urlopen(url)
    data = response.read()
    text = data.decode('utf-8') # a `str`;
    soup = BeautifulSoup(text,'xml')
    # Find the TitleText element with text "Purpose of Act"
    title = soup.find("ShortTitle")
    description = soup.find("LongTitle")

    if title and description:
        title = title.text
        description = description.text

    if not title and description:
        title = description.text
        description = description.text

    if not description and title:
        title = title.text
        description = ''

    

    heading = soup.find("Heading", text="Purpose of Act")
    if not heading :
       heading = soup.find("Heading", text="Purpose")
    purpose = ''
    if heading:
        # Get the next sibling, which should contain the text you want
        text_after_title = heading.find_next_sibling()
        all_text = text_after_title.find_all("Text")
        for i in all_text:
            purpose += i.text + ' '

    pattern = re.compile(r'[^a-zA-Z0-9]')
    
    path_to_file = os.path.join("path to folder") + re.sub(pattern, '', title[:50]) + '.json'

    if not os.path.isfile(path_to_file):
        print("writing " + title + '.json')
        with open((path_to_file), 'w') as f:
            json.dump({"title": title,"url": law_url,"description":description, "purpose": purpose}, f)

    else :
        print (title, "Already exists")
    


## dumping data to Neo4j

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    separators=["\n\n", "\n", ". ", " ", ""],
)

In [13]:
merge_chunk_node_query = """
MERGE(mergedChunk:UpdatedChunk {chunkId: $chunkParam.chunkId})
    ON CREATE SET 
        mergedChunk.chunkSeqId = $chunkParam.chunkSeqId, 
        mergedChunk.text = $chunkParam.text,
        mergedChunk.ActId = $chunkParam.ActId,
        mergedChunk.Federal = $chunkParam.Federal,
        mergedChunk.Regulation = $chunkParam.Regulation,
        mergedChunk.Act_url = $chunkParam.Act_url
RETURN mergedChunk
"""

In [14]:
create_embeddings = """
        MATCH (chunk:UpdatedChunk) WHERE
        chunk.chunkId = $chunkParam.chunkId
        AND chunk.chunkSeqId = $chunkParam.chunkSeqId
        AND chunk.ActId = $chunkParam.ActId
        AND chunk.Act_url = $chunkParam.Act_url
        AND chunk.text = $chunkParam.text
        AND chunk.Federal = $chunkParam.Federal
         AND chunk.Regulation = $chunkParam.Regulation
        AND chunk.textEmbedding is NULL
        CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", $vector)
        RETURN chunk
    """


In [15]:
def create_metadata(token_split_texts,title, act_url):
    chunks_with_metadata = [] # use this to accumlate chunk records
    chunk_seq_id = 0
    for chunk in token_split_texts: # only take the first 20 chunks
        #form_id = file[file.rindex('/') + 1:file.rindex('.')] # extract form id from file name
        # finally, construct a record with metadata and the chunk text
        chunks_with_metadata.append({
            'text': chunk, 
            # metadata from looping...
            'chunkSeqId': chunk_seq_id,
            'chunkId': f'{title}-chunk-{chunk_seq_id:04d}',
            'ActId': f'{title}',
            'Act_url': f'{act_url}',
            'Federal': True,
            'Regulation': True
            # constructed metadata...
            # metadata from file...
        })
        chunk_seq_id += 1
    return chunks_with_metadata

In [16]:
def create_chunks(item_text, title, act_url):
    item_text_chunks = text_splitter.split_text(item_text) # split the text into chunks
    token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=20, tokens_per_chunk=256)
    token_split_texts = []
    for text in item_text_chunks:
        token_split_texts += token_splitter.split_text(text)
    meta_data = create_metadata(token_split_texts,title, act_url)    
    return meta_data

In [17]:
def create_chunk_relationship(act_info):
    cypher = """
  MATCH (from_same_section:UpdatedChunk)
  WHERE from_same_section.ActId = $ActParam['ActId']
  AND from_same_section.Act_url = $ActParam['Act_url']
  WITH from_same_section
    ORDER BY from_same_section.chunkSeqId ASC
  WITH collect(from_same_section) as section_chunk_list
    CALL apoc.nodes.link(
        section_chunk_list, 
        "NEXT", 
        {avoidDuplicates: true}
    )  // NEW!!!
  RETURN size(section_chunk_list)
"""
    kg.query(cypher, params={'ActParam': act_info})

In [10]:
def create_chunk_embeddings(tokens):
    for chunk in tokens:
        # print(f"Creating `:Chunk` node for chunk ID {chunk['chunkSeqId']}")
        kg.query(merge_chunk_node_query, 
            params={
                'chunkParam': chunk
            })
        vector = embeddings.embed_query(chunk['text'])
        result = kg.query(create_embeddings, params={'chunkParam':chunk, 'vector':vector})
        if result:
            print("Embedding created")
        else:
            print("Embedding not created")
    create_chunk_relationship(tokens[0])       

In [18]:
def extract_data(json,  search=False):
    title = json['title']
    act_url = json['url']
    text = json['description'] + '\n ' + json['purpose']
    if json['description'] == "" and json['purpose'] == "":
        text = title
    print(title)
    token = create_chunks(text, title, act_url)
    found = create_chunk_embeddings(token)

In [19]:
import time
from tqdm import tqdm
from threading import Thread
folder_path = "Full Path to the folder"

num_files = len(os.listdir(folder_path))  # Count files for progress bar

with tqdm(total=num_files, desc="Processing Files") as pbar:
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r") as file:
            data = json.load(file)
            extract_data(data)
        pbar.update(1)


Processing Files:   0%|          | 0/1721 [00:00<?, ?it/s]

Fort William First Nation Sawmill Regulations


Processing Files:   0%|          | 1/1721 [00:01<56:00,  1.95s/it]

Embedding created
Canadian Human Rights Benefit Regulations


Processing Files:   0%|          | 2/1721 [00:03<53:52,  1.88s/it]

...

Embedding created
CPTPP Rules of Origin Regulations


Processing Files: 100%|█████████▉| 1720/1721 [1:01:34<00:02,  2.22s/it]

Embedding created
Diversion of Imported Goods Exemption Regulations


Processing Files: 100%|██████████| 1721/1721 [1:01:36<00:00,  2.15s/it]

Embedding created
